In [48]:
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [49]:
# Loading data
pd.set_option('display.max_columns', None)
mental_df = pd.read_csv('./Resources/cleaned_data.tsv', sep='\t')
mental_df

,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,Does your employer offer resources to learn more about mental health concerns and options for seeking help?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you bring up a mental health issue with a potential employer in an interview?,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your gender?,What country do you live in?,What country do you work in?
0,No,Yes,Yes,Somewhat easy,No,No,Yes,No,1,"Yes, they all did",I was aware of some,None did,"Yes, always",None of them,Some of my previous employers,Some did,None of them,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,Yes,Yes,Yes,Yes,1,Rarely,Sometimes,Male,United States of America,United States of America
1,Yes,Yes,No,Neither easy nor difficult,Yes,Maybe,No,No,1,I don't know,N/A (not currently aware),Some did,I don't know,Some of them,Some of my previous employers,Some did,Some of them,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,1,Sometimes,Sometimes,Female,United States of America,United States of America
2,Yes,I am not sure,Yes,Somewhat easy,Yes,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,I don't know,"Yes, all of them","No, at none of my previous employers",None did,Some of them,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,Yes,No,1,Not applicable to me,Often,Male,United Kingdom,United Kingdom
3,I don't know,No,No,Somewhat easy,No,No,Yes,No,1,Some did,I was aware of some,None did,I

In [50]:
# Generate our categorical variable list
mental_cat = mental_df.dtypes[mental_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
mental_df[mental_cat].nunique()

Does your employer provide mental health benefits as part of healthcare coverage?                                                               4
Do you know the options for mental health care available under your employer-provided coverage?                                                 3
Does your employer offer resources to learn more about mental health concerns and options for seeking help?                                     3
If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:                                     6
Do you think that discussing a mental health disorder with your employer would have negative consequences?                                      3
Do you think that discussing a physical health issue with your employer would have negative consequences?                                       3
Do you feel that your employer takes mental health as seriously as physical health?                                         

In [52]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mental_df[mental_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(mental_cat)
encode_df.head()

,Does your employer provide mental health benefits as part of healthcare coverage?_I don't know,Does your employer provide mental health benefits as part of healthcare coverage?_No,Does your employer provide mental health benefits as part of healthcare coverage?_Not eligible for coverage / N/A,Does your employer provide mental health benefits as part of healthcare coverage?_Yes,Do you know the options for mental health care available under your employer-provided coverage?_I am not sure,Do you know the options for mental health care available under your employer-provided coverage?_No,Do you know the options for mental health care available under your employer-provided coverage?_Yes,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_I don't know,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_No,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_Yes,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_I don't know","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Neither easy nor difficult","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Somewhat difficult","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Somewhat easy","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Very difficult","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Very easy",Do you think that discussing a mental health disorder with your employer would have negative consequences?_Maybe,Do you think that discussing a mental health disorder with your employer would have negative consequences?_No,Do you think that discussing a mental health disorder with your employer would have negative consequences?_Yes,Do you think that discussing a physical health issue with your employer would have negative consequences?_Maybe,Do you think that discussing a physical health issue with your employer would have negative consequences?_No,Do you think that discussing a physical health issue with your employer would have negative consequences?_Yes,Do you feel that your employer takes mental health as seriously as physical health?_I don't know,Do you feel that your employer takes mental health as seriously as physical health?_No,Do you feel that your employer takes mental health as seriously as physical health?_Yes,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?_No,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?_Yes,Have your previous employers provided mental health benefits?_I don't know,"Have your previous employers provided mental health benefits?_No, none did",Have your previous employers provided mental health benefits?_Some did,"Have your previous employers provided mental health benefits?_Yes, they all did",Were you aware of the options for mental health care provided by your previous employers?_I was aware of some,Were you aware of the options for mental health care provided by your previous employers?_N/A (not currently aware),"Were you aware of the options for mental health care provided by your previous employers?_No, I only became aware later","Were you aware of the options for mental health care provided by your previous employers?_Yes, I was aware of all of them",Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?_I don't know,Did your previous employers ever formally discuss mental health (as part of a we

In [53]:
# Merge one-hot encoded features and drop the originals
mental_df = mental_df.merge(encode_df,left_index=True, right_index=True)
mental_df = mental_df.drop(mental_cat,1)
mental_df.head()

,Do you have previous employers?,Have you ever sought treatment for a mental health issue from a mental health professional?,Does your employer provide mental health benefits as part of healthcare coverage?_I don't know,Does your employer provide mental health benefits as part of healthcare coverage?_No,Does your employer provide mental health benefits as part of healthcare coverage?_Not eligible for coverage / N/A,Does your employer provide mental health benefits as part of healthcare coverage?_Yes,Do you know the options for mental health care available under your employer-provided coverage?_I am not sure,Do you know the options for mental health care available under your employer-provided coverage?_No,Do you know the options for mental health care available under your employer-provided coverage?_Yes,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_I don't know,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_No,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_Yes,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_I don't know","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Neither easy nor difficult","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Somewhat difficult","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Somewhat easy","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Very difficult","If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:_Very easy",Do you think that discussing a mental health disorder with your employer would have negative consequences?_Maybe,Do you think that discussing a mental health disorder with your employer would have negative consequences?_No,Do you think that discussing a mental health disorder with your employer would have negative consequences?_Yes,Do you think that discussing a physical health issue with your employer would have negative consequences?_Maybe,Do you think that discussing a physical health issue with your employer would have negative consequences?_No,Do you think that discussing a physical health issue with your employer would have negative consequences?_Yes,Do you feel that your employer takes mental health as seriously as physical health?_I don't know,Do you feel that your employer takes mental health as seriously as physical health?_No,Do you feel that your employer takes mental health as seriously as physical health?_Yes,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?_No,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?_Yes,Have your previous employers provided mental health benefits?_I don't know,"Have your previous employers provided mental health benefits?_No, none did",Have your previous employers provided mental health benefits?_Some did,"Have your previous employers provided mental health benefits?_Yes, they all did",Were you aware of the options for mental health care provided by your previous employers?_I was aware of some,Were you aware of the options for mental health care provided by your previous employers?_N/A (not currently aware),"Were you aware of the options for mental health care provided by your previous employers?_No, I only became aware later","Were you aware of the options for mental health care provided by your previous employers?_Yes, I was aware of all of them",Did your previous employers ever formally discuss mental health (as part of a wellness campaign or 

In [60]:
column_rename = {}
count = 0
for column in mental_df.columns:
    
    column_rename[column] = count
    count = count + 1
    
    
column_rename
renamed_df = mental_df.rename(columns=column_rename)

In [44]:
dict = {}
count = 0
for column in mental_df.columns:
    
    dict[count] = [column]
    count = count + 1
    
    
dict

questions_df = pd.DataFrame.from_dict(dict, orient='index', columns=['Question'])

questions_df

,Question
0,Do you have previous employers?
1,Have you ever sought treatment for a mental he...
2,Does your employer provide mental health benef...
3,Does your employer provide mental health benef...
4,Does your employer provide mental health benef...
...,...
108,What country do you live in?_United Kingdom
109,What country do you live in?_United States of ...
110,What country do you work in?_Other
111,What country do you work in?_United Kingdom


In [45]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///test.db', echo=False)

In [58]:
questions_df.to_sql('questions', con=engine, if_exists='replace')
engine.execute("SELECT * FROM questions").fetchall()

[(0, 'Do you have previous employers?'),
 (1, 'Have you ever sought treatment for a mental health issue from a mental health professional?'),
 (2, "Does your employer provide mental health benefits as part of healthcare coverage?_I don't know"),
 (3, 'Does your employer provide mental health benefits as part of healthcare coverage?_No'),
 (4, 'Does your employer provide mental health benefits as part of healthcare coverage?_Not eligible for coverage / N/A'),
 (5, 'Does your employer provide mental health benefits as part of healthcare coverage?_Yes'),
 (6, 'Do you know the options for mental health care available under your employer-provided coverage?_I am not sure'),
 (7, 'Do you know the options for mental health care available under your employer-provided coverage?_No'),
 (8, 'Do you know the options for mental health care available under your employer-provided coverage?_Yes'),
 (9, "Does your employer offer resources to learn more about mental health concerns and options for seekin

In [61]:
renamed_df.to_sql('data', con=engine, if_exists='replace')
engine.execute("SELECT * FROM data").fetchall()

[(0, 1, 1, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0),
 (1, 1, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.

In [ ]:
# Define the target set.
y = mental_df["Have you been diagnosed with a mental health condition by a medical professional?_Yes"]

# Define the features set.
X = mental_df.drop(columns=["Have you been diagnosed with a mental health condition by a medical professional?_Yes","Have you been diagnosed with a mental health condition by a medical professional?_No"])

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [ ]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test)
predictions

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

In [ ]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)